# **Maintenance des Segments**

Il s'agit ici de proposer des périodes de maintenance, donc des périodes au dessus desquelles le modèle est à la dérive c'est-à-dire ne parvient plus à prédire des segments stables des observations.

Il s'agira alors de :
- **Déterminer à quel moment la performance tombe sous un seuil critique**
- **En déduire la durée de validité du modèle → donc la fréquence de maintenance**
